In [ ]:
!pip install --proxy 10.0.0.1:3128 numpy

 --upgrade pip nltk gensim pandas pymorphy2 emoji numpy

# Статистика комментариев и использования в них эмодзи для сообществ социальной сети ВК по Алтайскому краю

### Импорт необходимых библиотек

In [1]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import zipfile
import numpy as np

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

In [6]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### Подготовка датасета

##### Чтение датасета

In [7]:
colnames = ['index', 'comment']

In [8]:
comments = pd.read_csv('datasets/barneos22.csv', names=colnames, header=None)

In [9]:
comments

,index,comment
0,NaN,0
1,0.0,Ледокол 🤣
2,1.0,Красавец! Какая же удивительная природа😊 столь...
3,2.0,"[id457697599|Анастасия], да какие это люди? Ма..."
4,3.0,Шикарный👍
...,...,...
1722,1721.0,"[id255548341|Анастасия], люди по умнее, кому д..."
1723,1722.0,"[id258410293|Евгений], глупости говорите. У ме..."
1724,1723.0,На видео министр здравоохранения Попов смешно ...
1725,1724.0,"Огласите пожалуйста весь список, все 700 учреж..."


In [10]:
comments = comments.drop(labels = [0],axis = 0)

In [11]:
comments = comments.drop(columns = ['index'],axis = 1)

In [12]:
comments = comments.reset_index(drop=True)

In [13]:
comments

,comment
0,Ледокол 🤣
1,Красавец! Какая же удивительная природа😊 столь...
2,"[id457697599|Анастасия], да какие это люди? Ма..."
3,Шикарный👍
4,"""краснокнижного"" )"
...,...
1721,"[id255548341|Анастасия], люди по умнее, кому д..."
1722,"[id258410293|Евгений], глупости говорите. У ме..."
1723,На видео министр здравоохранения Попов смешно ...
1724,"Огласите пожалуйста весь список, все 700 учреж..."


##### Предобработка комментариев

Удаление из комментраиев-ответов имени и идентификатора пользователя [id255548341|Анастасия], которому адресован ответ 

In [14]:
pattern = r'\[(.*?)\]'

def reply_delete(doc):
    doc = re.sub(pattern, ' ', str(doc))
    return doc

In [15]:
comments['comment'] = comments['comment'].apply(reply_delete)

In [16]:
comments

,comment
0,Ледокол 🤣
1,Красавец! Какая же удивительная природа😊 столь...
2,", да какие это люди? Маленько не дотягивают"
3,Шикарный👍
4,"""краснокнижного"" )"
...,...
1721,", люди по умнее, кому дорого свое здоровье, е..."
1722,", глупости говорите. У меня не одному знакомо..."
1723,На видео министр здравоохранения Попов смешно ...
1724,"Огласите пожалуйста весь список, все 700 учреж..."


In [17]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    doc = re.sub(patterns, ' ', str(doc))
    for c in str(doc):
      if emoji.is_emoji(c):
          i = doc.index(c)
          doc = doc[:i] + doc[i:]
    tokens = []
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
        elif emoji.is_emoji(token):
            tokens.append(token)
    return tokens

In [18]:
comments['comment'] = comments['comment'].apply(lemmatize)

In [20]:
comments = comments.dropna()

In [21]:
comments

,comment
0,"[ледокол, 🤣]"
1,"[красавец, какой, удивительный, природа😊, стол..."
2,"[какой, это, человек, маленько, дотягивать]"
3,[шикарный👍]
4,[краснокнижный]
...,...
1721,"[человек, умный, кто, дорого, свой, здоровье, ..."
1722,"[глупость, говорить, у, один, знакомый, родств..."
1723,"[на, видео, министр, здравоохранение, попов, с..."
1724,"[огласить, пожалуйста, весь, список, учреждение]"


##### Проверка на наличие эмодзи в полученном датасете комментариев

In [22]:
total = comments.count()[0]

In [23]:
total

1726

In [24]:
def split(s):
    return [char for char in s]


def check(comment):
    flag=0
    for item in comment:
        for c in split(item):
            if emoji.is_emoji(c):
                flag+=1
    if flag > 0:
        return(comment)
    return None 

In [25]:
comments['comment'] = comments['comment'].apply(check)

In [26]:
comments = comments.dropna()

In [27]:
comments

,comment
0,"[ледокол, 🤣]"
1,"[красавец, какой, удивительный, природа😊, стол..."
3,[шикарный👍]
8,"[каков, красавец, 😍]"
9,"[зачем, ❓, собраться, убивать, это, тот, стрел..."
...,...
1676,"[здравствуйте, просить, спешить, принимать, ре..."
1689,"[взрослый, мальчик, конец, март, 🤣😉]"
1697,[🙃]
1720,"[в, барнаул, хороший, краевой, больница, многи..."


#### Подсчет статистики использования эмодзи

In [28]:
comments.count()[0]

300

In [32]:
comments.count()[0]/total

0.17381228273464658

#### Поверка на сообщения, состоящие только из СГИ

In [33]:
total_includes_emoji = comments.count()[0]

In [34]:
def only_emoji(doc):
    count_token=0
    count_doc=0    
    for token in doc:
        for c in split(token):
            if emoji.is_emoji(c):
                count_token+=1
        if count_token==len(token):
            count_doc+=1
    if count_doc==len(doc):
        return(doc)
    return None

In [35]:
comments['comment'] = comments['comment'].apply(only_emoji)

C:\Users\Admin\AppData\Local\Temp\ipykernel_18680\4091109362.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments['comment'] = comments['comment'].apply(only_emoji)


In [37]:
comments = comments.dropna()

In [39]:
comments

,comment
215,[🤣]
370,[💯]
393,[🤐]
455,[👍]
470,[😄]
569,[😅]
648,[🤣]
711,[🤨]
726,[👍]
747,[🤣👍]


In [43]:
comments.count()[0]/total

0.019119351100811123

In [44]:
comments.count()[0]/total_includes_emoji

0.11